In [ ]:
### Check deleted workflow runs for output files and delete them

from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

wfrs = ff_utils.get_metadata('workflow-runs-sbg' , connection=ff)['@graph']
print len(wfrs), 'workflowruns in the system'
counter = 0
deleted_output = 0
for a_wfr in wfrs:
    counter += 1
    if counter % 100 == 0:
        print counter, deleted_output

    if a_wfr['status'] == 'deleted':
        outputs = [i.get('value') for i in a_wfr['output_files'] if i.get('value')]
    else:
        continue  
    if outputs:
        for out_file in outputs:
            file_status = ff_utils.get_metadata(out_file, connection=ff, frame='raw')['status']
            print file_status
            if file_status != 'deleted':
                deleted_output += 1
                ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,connection=ff)
print deleted_output, 'output files deleted'
                           